In [3]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


Note: you may need to restart the kernel to use updated packages.


In [14]:
import sys

print(sys.executable)

C:\Users\vansh\anaconda3\python.exe


# CNN Final Multi_Model Fusion

In [8]:
# ==========================================
# 1. IMPORT LIBRARIES
# ==========================================
import numpy as np
import pandas as pd
import tensorflow as tf
import os

from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


# ==========================================
# 2. LOAD & CLEAN CSV DATA (CRITICAL FIX)
# ==========================================
df = pd.read_csv(
    r"C:\Users\vansh\Desktop\satellite imagery\project_root\final_multimodal.csv"
)

# 🔥 DROP ROWS WHERE IMAGE IS MISSING
df = df.dropna(subset=["image_path"]).reset_index(drop=True)

# Ensure image paths are strings
df["image_path"] = df["image_path"].astype(str)

print("Dataset shape after cleaning:", df.shape)


# ==========================================
# 3. SELECT FEATURES & TARGET
# ==========================================
tabular_features = [
    "bedrooms", "bathrooms", "sqft_living", "floors",
    "waterfront", "view", "condition", "grade",
    "sqft_above", "sqft_basement",
    "lat", "long", "sqft_living15"
]

X_tab = df[tabular_features].values
image_paths = df["image_path"].values

# LOG TRANSFORM TARGET (BEST PRACTICE)
y = np.log1p(df["price"].values)


# ==========================================
# 4. TRAIN–TEST SPLIT
# ==========================================
X_tab_train, X_tab_test, img_train, img_test, y_train, y_test = train_test_split(
    X_tab,
    image_paths,
    y,
    test_size=0.2,
    random_state=42
)


# ==========================================
# 5. SCALE TABULAR FEATURES
# ==========================================
scaler = StandardScaler()
X_tab_train = scaler.fit_transform(X_tab_train)
X_tab_test = scaler.transform(X_tab_test)


# ==========================================
# 6. LOAD & PREPROCESS IMAGES
# ==========================================
IMG_SIZE = (224, 224)

def load_and_preprocess_image(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=IMG_SIZE)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img

X_img_train = np.array([load_and_preprocess_image(p) for p in img_train])
X_img_test  = np.array([load_and_preprocess_image(p) for p in img_test])

print("Image train shape:", X_img_train.shape)
print("Tabular train shape:", X_tab_train.shape)


# ==========================================
# 7. IMAGE BRANCH (CNN)
# ==========================================
cnn_base = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

cnn_base.trainable = False

img_input = tf.keras.Input(shape=(224, 224, 3))
x = cnn_base(img_input)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation="relu")(x)


# ==========================================
# 8. TABULAR BRANCH
# ==========================================
tab_input = tf.keras.Input(shape=(X_tab_train.shape[1],))
t = layers.Dense(64, activation="relu")(tab_input)
t = layers.Dense(32, activation="relu")(t)


# ==========================================
# 9. FUSION
# ==========================================
combined = layers.Concatenate()([x, t])
z = layers.Dense(64, activation="relu")(combined)
z = layers.Dense(32, activation="relu")(z)
output = layers.Dense(1)(z)  # predicts log(price)

model = models.Model(
    inputs=[img_input, tab_input],
    outputs=output
)

model.compile(
    optimizer="adam",
    loss="mse"
)

model.summary()


# ==========================================
# 10. TRAIN MODEL
# ==========================================
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    [X_img_train, X_tab_train],
    y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)


# ==========================================
# 11. EVALUATION (CORRECT METRICS)
# ==========================================
# Predict in LOG SPACE
y_pred_log = model.predict([X_img_test, X_tab_test]).flatten()

# RMSE in ORIGINAL PRICE SCALE
y_pred_price = np.expm1(y_pred_log)
y_true_price = np.expm1(y_test)

rmse = np.sqrt(mean_squared_error(y_true_price, y_pred_price))

# R² in LOG SPACE (CORRECT)
r2 = r2_score(y_test, y_pred_log)

print("\nMULTIMODAL MODEL RESULTS (FINAL)")
print("RMSE (price scale):", rmse)
print("R2 Score (log-price space):", r2)


Dataset shape after cleaning: (499, 16)
Image train shape: (399, 224, 224, 3)
Tabular train shape: (399, 13)
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 118s 1us/step


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ resnet50 (Functional)         │ (None, 7, 7, 2048)        │      23,587,712 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_5 (InputLayer)    │ (None, 13)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling2d      │ (None, 2048)              │               0 │ resnet50[0][0]             │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_10 (Dense)              │ (None, 64)                │             896 │ input_layer_5[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 128)               │         262,272 │ global_average_pooling2d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_11 (Dense)              │ (None, 32)                │           2,080 │ dense_10[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 160)               │               0 │ dense_9[0][0],             │
│                               │                           │                 │ dense_11[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_12 (Dense)              │ (None, 64)                │          10,304 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_13 (Dense)              │ (None, 32)                │           2,080 │ dense_12[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_14 (Dense)              │ (None, 1)                 │              33 │ dense_13[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 23,865,377 (91.04 MB)

 Trainable params: 277,665 (1.06 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - loss: 22.8843 - val_loss: 3.5328
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - loss: 2.6293 - val_loss: 1.6056
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - loss: 1.1128 - val_loss: 0.6738
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - loss: 0.5892 - val_loss: 0.5915
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - loss: 0.4075 - val_loss: 0.5602
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 0.3504 - val_loss: 0.5855
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.3435 - val_loss: 0.6325
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.2451 - val_loss: 0.5450
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - loss: 0.2071 - val_loss: 0.5438
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - loss: 0.1729 - val_loss: 0.5691
Epoch 11/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - loss: 0.1537 - val_loss: 0.5431
Epoch 12/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - loss: 0.1331 - val_l